In [4]:
import matplotlib.pyplot as plt

class Sketcher:
    def __init__(self, windowname, dests, colors_func):
        self.prev_pt = None
        self.windowname = windowname
        self.dests = dests
        self.colors_func = colors_func
        self.dirty = False
        self.show()
        cv2.setMouseCallback(self.windowname, self.on_mouse)
        #cv2.setMouseCallback(None, self.on_mouse)

    def show(self):
        # cv2.imshow(self.windowname, self.dests[0])
        # cv2.imshow(self.windowname + ": mask", self.dests[1])

        # cv2_imshow(self.windowname, self.dests[0])
        # cv2_imshow(self.windowname + ": mask", self.dests[1])

        plt.imshow(self.windowname, cv2.cvtColor(self.dests[0], cv2.COLOR_BGR2RGB))
        #plt.imshow(self.windowname, self.dests[0])
        plt.show()
        #plt.imshow(self.dests[0])
        plt.imshow(self.windowname + ": mask", cv2.cvtColor(self.dests[1], cv2.COLOR_BGR2RGB))
        #plt.imshow(self.windowname + ": mask", self.dests[1])
        plt.show()
        #plt.imshow(self.dests[1])
        
        
        #image1=(self.windowname,self.dests[0])
        #cv2_imshow(image1)
        #image2=(self.windowname + ": mask", self.dests[1])
        #cv2_imshow(image2)

        #cv2_imshow(self.windowname) 
        #cv2_imshow(self.dests[0])
        #cv2_imshow(self.windowname+":mask")
        #cv2_imshow(self.dests[1])


    # onMouse function for Mouse Handling
    def on_mouse(self, event, x, y, flags, param):
        pt = (x, y)
        if event == cv2.EVENT_LBUTTONDOWN:
            self.prev_pt = pt
        elif event == cv2.EVENT_LBUTTONUP:
            self.prev_pt = None

        if self.prev_pt and flags & cv2.EVENT_FLAG_LBUTTON:
            for dst, color in zip(self.dests, self.colors_func()):
                cv2.line(dst, self.prev_pt, pt, color, 10)
            self.dirty = True
            self.prev_pt = pt
            self.show()


SyntaxError: positional argument follows keyword argument (<ipython-input-4-335fe21b9b98>, line 21)

In [5]:
import cv2
import numpy as np

#from Sketcher import Sketcher
from libs.util import MaskGenerator, ImageChunker
from libs.pconv_model import PConvUnet

import sys
from copy import deepcopy

import matplotlib.pyplot as plt

print('load model...')
model = PConvUnet(vgg_weights=None, inference_only=True)
model.load('pconv_imagenet.h5', train_bn=False)
# model.summary()

#img = plt.imread(sys.argv[1], cv2.IMREAD_COLOR)
#img = plt.imread('d', cv2.IMREAD_COLOR)
img = cv2.imread('data/images/04.jpg', cv2.IMREAD_COLOR)


img_masked = img.copy()
mask = np.zeros(img.shape[:2], np.uint8)

sketcher = Sketcher('image', [img_masked, mask], lambda : ((255, 255, 255), 255))
chunker = ImageChunker(512, 512, 30)

while True:
  key = cv2.waitKey()

  if key == ord('q'): # quit
    break
  if key == ord('r'): # reset
    print('reset')
    img_masked[:] = img
    mask[:] = 0
    sketcher.show()
  if key == 32: # hit spacebar to run inpainting
    input_img = img_masked.copy()
    input_img = input_img.astype(np.float32) / 255.

    input_mask = cv2.bitwise_not(mask)
    input_mask = input_mask.astype(np.float32) / 255.
    input_mask = np.repeat(np.expand_dims(input_mask, axis=-1), repeats=3, axis=-1)

    # cv2.imshow('input_img', input_img)
    # cv2.imshow('input_mask', input_mask)

    print('processing...')

    chunked_imgs = chunker.dimension_preprocess(deepcopy(input_img))
    chunked_masks = chunker.dimension_preprocess(deepcopy(input_mask))

    # for i, im in enumerate(chunked_imgs):
    #   cv2.imshow('im %s' % i, im)
    #   cv2.imshow('mk %s' % i, chunked_masks[i])

    pred_imgs = model.predict([chunked_imgs, chunked_masks])
    result_img = chunker.dimension_postprocess(pred_imgs, input_img)

    print('completed!')

    #cv2.imshow('result', result_img)
    cv2_imshow('result', result_img)

cv2.destroyAllWindows()


load model...


TypeError: unhashable type: 'numpy.ndarray'